# Подсчет суммарных токов по компартментам через `Python`

In [1]:
%pylab inline
style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


In [2]:
from neuron import h, gui

In [3]:
h.load_file('Spencerian-ab.hoc')

1.0

In [4]:
h.run()

0.0

Можно доставать суммарный К и Na токи:

In [5]:
h.dendrites[0](0.5).ik

0.0029152034477829855

In [6]:
h.soma(0.5).ik ==  h.soma(0.5).kltab.ik + h.soma(0.5).khtab.ik

True

А также площадь в мкм^2 (на самом деле, это и из `hoc` можно)

In [7]:
x = h.dendrites[0](0.5)

In [8]:
x.area(),

(23.32865831873609,)

In [9]:
allclose(x.area(), pi*x.diam*h.dendrites[0].L/h.dendrites[0].nseg)

True

In [10]:
from collections import defaultdict

In [11]:
# TODO: во все компартменты добавить *все* токи, только ненужные токи "выключить",
#       выставив gbar=0

def net_dendrite_currents():
    "Total dendrite currents in nA"
    net_i = defaultdict(lambda: 0)

    for i in range(int(h.Ndend)):
        dend = h.dendrites[i]
        for loc in dend:
            a = loc.area()*1e-8*1e6 #конвертируем площадь в см^2 и ток в nA заодно
            net_i['ik'] += loc.ik*a 
            net_i['ih'] += loc.ihsab.ih*a
            net_i['ih'] += loc.hcno.ih*a  # just in case I'll want to add this
            net_i['ileak'] += loc.i_leak*a
    return net_i

def net_soma_currents():
    net_i = defaultdict(lambda: 0)
    loc = h.soma(0.5)
    a = loc.area()*1e-8*1e6 #конвертируем площадь в см^2 и ток в nA заодно
    net_i['ik'] += loc.ik*a # все К токи
    net_i['ina'] += loc.ina*a
    net_i['ih'] += loc.ihsab.ih*a
    net_i['ih'] += loc.hcno.ih*a
    net_i['ileak'] += loc.i_leak*a
    return net_i

def net_axon_currents():
    net_i = defaultdict(lambda: 0)
    for comp in (h.ax_init, h.ax_stub):
        for loc in comp:
            a = loc.area()*1e-8*1e6 #конвертируем площадь в см^2 и ток в nA заодно
            #net_i['ik'] += loc.ik*a # все К токи
            #net_i['ih'] += loc.ihsab.ih*a
            #net_i['ih'] += loc.hcno.ih*a
            net_i['ileak'] += loc.i_leak*a
            net_i['ina'] += loc.ina*a
    return net_i

In [12]:
di = net_dendrite_currents()
si = net_soma_currents()
ai = net_axon_currents()

In [13]:
print di, si, ai

defaultdict(<function <lambda> at 0x7f53460c9140>, {'ih': -0.41540706631225116, 'ik': 0.2764096723748255, 'ileak': 0.2270436210049071}) defaultdict(<function <lambda> at 0x7f53460c9050>, {'ih': -1.0618298082554412, 'ik': 0.9075170983290506, 'ina': 0.0, 'ileak': 0.05613426969447634}) defaultdict(<function <lambda> at 0x7f5374228c08>, {'ileak': 0.010602679603654252, 'ina': -0.0004955310161020326})


In [14]:
curs = ['ik','ina', 'ileak', 'ih']
totals = [[s[c] for c in curs] for s in [di,si,ai]]

In [15]:
print totals
print u"Сумма:", sum(totals)

[[0.2764096723748255, 0, 0.2270436210049071, -0.41540706631225116], [0.9075170983290506, 0.0, 0.05613426969447634, -1.0618298082554412], [0, -0.0004955310161020326, 0.010602679603654252, 0]]
Сумма: -2.50645768805e-05


=> В стационарном состоянии сумма всех токов близка к нулю.